In [210]:
from keras.losses import categorical_crossentropy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import MaxPooling2D, Flatten
import pandas as pd
from datetime import datetime as dt
import talib
import numpy as np

In [211]:
data_df = pd.read_csv('./225537_daily.csv', index_col=[0])

In [212]:
data_df.head()

,open,high,low,close,volume,sma_5,sma_10,ema_20,mtm6_mtm12,fastk,...,roc_10,bband_upper,bband_middle,bband_lower,macd,macdsignal,macdhist,adosc,cci_14,atr_14
date,,,,,,,,,,,,,,,,,,,,,
2008-10-28,449.0,459.0,413.05,421.40,26950.0,447.50,457.475,471.137197,0.909154,0.000000,...,-7.506585,492.770388,447.50,402.229612,-24.489724,-23.933145,-0.556578,186580.861422,-84.887688,42.672054
2008-10-29,430.0,438.5,404.00,416.20,1254388.0,434.70,454.805,465.905083,0.977321,0.000000,...,-6.028449,471.064378,434.70,398.335622,-25.928546,-24.332225,-1.596321,25451.757073,-109.706281,42.088336
2008-10-31,425.0,464.9,419.00,429.90,562630.0,427.56,452.655,462.476028,1.077911,89.362669,...,-4.762960,446.874078,427.56,408.245922,-25.667469,-24.599274,-1.068195,-136773.025365,-45.957285,42.560598
2008-11-03,436.0,436.0,398.15,403.00,853239.0,419.24,448.110,456.811644,1.025402,0.000000,...,-10.134909,437.852082,419.24,400.627918,-27.316284,-25.142676,-2.173608,-392614.143495,-129.301056,42.224126
2008-11-04,407.0,419.0,395.00,401.25,899546.0,414.35,439.765,451.520059,1.086997,0.000000,...,-17.216835,436.174023,414.35,392.525977,-28.436395,-25.801420,-2.634975,-598730.915519,-129.921943,40.922403


In [213]:
#data_df['date'] = data_df['date'].map(lambda x: dt.strptime(x, '%m/%d/%Y %I:%M:%S %p').date())
data_df['volume'] = data_df['volume'].astype(float)
#data_df = data_df.set_index('date')

# Simple Moving Average
data_df['sma_5'] = talib.SMA(data_df['close'].values, timeperiod=5)
data_df['sma_10'] = talib.SMA(data_df['close'].values, timeperiod=10)
# Exponential Moving Average
data_df['ema_20'] = talib.EMA(data_df['close'].values, timeperiod=20)
# Momentum 6 Month / Momentum 12 Month
data_df['mtm6_mtm12'] = talib.MOM(data_df['close'].values, timeperiod=126)/talib.MOM(data_df['close'].values, 
                                  timeperiod=252)
# Stochastic Relative Strength Index
data_df['fastk'], data_df['fastd'] = talib.STOCHRSI(data_df['close'].values, timeperiod=14, fastk_period=5,
                                                    fastd_period=3, fastd_matype=0)
# Rate Of Change
data_df['roc_10'] = talib.ROC(data_df['close'].values, timeperiod=10)
# Bollinger Bands
data_df['bband_upper'], data_df['bband_middle'], data_df['bband_lower'] = talib.BBANDS(data_df['close'].values,
                                                                                     timeperiod=5, nbdevup=2, nbdevdn=2,
                                                                                     matype=0)
# Moving Average Convergence Divergence
data_df['macd'], data_df['macdsignal'], data_df['macdhist'] = talib.MACD(data_df['close'].values, fastperiod=12, 
                                                                         slowperiod=26, signalperiod=9)
# Chaikin A/D Oscillator
data_df['adosc'] = talib.ADOSC(data_df['high'], data_df['low'], data_df['close'], data_df['volume'], fastperiod=3,
                             slowperiod=10)
# Commodity Channel Index
data_df['cci_14'] = talib.CCI(data_df['high'].values, data_df['low'].values, data_df['close'].values, timeperiod=14)
# Average True Range
data_df['atr_14'] = talib.ATR(data_df['high'].values, data_df['low'].values, data_df['close'].values, timeperiod=14)
# Target
data_df['target'] = np.append(data_df['close'][1:].values, [np.nan])
# Drop Rows With NA Values In Any Column
data_df = data_df.dropna(axis=0, how='any')
# Popping The Target Column
target = data_df.pop('target').values

In [214]:
data_df.head()

,open,high,low,close,volume,sma_5,sma_10,ema_20,mtm6_mtm12,fastk,...,roc_10,bband_upper,bband_middle,bband_lower,macd,macdsignal,macdhist,adosc,cci_14,atr_14
date,,,,,,,,,,,,,,,,,,,,,
2009-11-13,1107.00,1120.95,1096.00,1108.65,162748.0,1107.48,1078.780,1037.040428,0.758821,0.000000,...,9.097619,1132.514824,1107.48,1082.445176,51.542123,44.986421,6.555701,82120.478264,85.592224,41.284589
2009-11-16,1110.00,1124.60,1080.50,1102.35,711035.0,1107.98,1087.875,1043.260388,0.770677,0.000000,...,8.992486,1132.479437,1107.98,1083.480563,50.400971,46.069331,4.331640,66997.143935,68.922981,41.485690
2009-11-17,1109.90,1119.80,1095.05,1110.75,511877.0,1111.88,1095.500,1049.687970,0.731145,14.514965,...,7.370710,1129.813388,1111.88,1093.946612,49.602622,46.775989,2.826632,98480.693987,71.206147,40.290283
2009-11-18,1110.75,1124.90,1103.05,1107.65,319970.0,1107.63,1099.815,1055.208163,0.731853,5.884116,...,4.053546,1113.280345,1107.63,1101.979655,48.164569,47.053705,1.110863,43465.695631,69.428519,38.973120
2009-11-19,1106.00,1107.65,1082.30,1098.90,365055.0,1105.66,1102.630,1059.369290,0.671755,0.000000,...,2.628998,1114.398970,1105.66,1096.921030,45.790999,46.801164,-1.010165,52976.883737,26.546429,38.000040


In [215]:
def multivariate_data(dataset, target, start_index, end_index, history_size, target_size, step, single_step=False):
    data = []
    labels = []
    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size
    for i in range(start_index, end_index):
        indices = range(i-history_size, i, step)
        data.append(dataset[indices])
        if single_step:
            labels.append(target[i+target_size])
        else:
            labels.append(target[i:i+target_size])
    return(np.array(data), np.array(labels))

In [216]:
TRAIN_SPLIT = round(0.8 * len(data_df))
# Normalizing Data
dataset = data_df.values
data_mean = dataset[:TRAIN_SPLIT].mean(axis=0)
data_std = dataset[:TRAIN_SPLIT].std(axis=0)
dataset = (dataset-data_mean)/data_std

In [217]:
past_history = 21
future_target = 1
STEP = 1
X_train, y_train = multivariate_data(dataset, target, 0, TRAIN_SPLIT, past_history, future_target, STEP, 
                                     single_step=True)
X_test, y_test = multivariate_data(dataset, target, TRAIN_SPLIT, None, past_history, future_target, STEP, 
                                     single_step=True)
X_train = X_train.reshape(X_train.shape[0],21,21,1)
X_test = X_test.reshape(X_test.shape[0],21,21,1)

In [218]:
model = Sequential()

In [219]:
model.add(Conv2D(32, (3, 3), padding="same", activation="relu", input_shape=(21, 21,1)))
model.add(Conv2D(64, (3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size =(2, 2)))
model.add(Flatten())
model.add(Dense(units = 128, activation = "relu"))
model.add(Dense(units = 64, activation = "relu"))
model.add(Dense(units=1, activation='relu'))

In [220]:
model.compile(loss="mae", optimizer="adam")

In [ ]:
BATCH_SIZE = 32
EPOCHS = 1000

model.fit(X_train, y_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          verbose=1,
          validation_data=(X_test, y_test))

Train on 1945 samples, validate on 469 samples
Epoch 1/1000
1945/1945 [==============================] - 4s 2ms/step - loss: 1170.0254 - val_loss: 598.7820
Epoch 2/1000
1945/1945 [==============================] - 4s 2ms/step - loss: 420.3897 - val_loss: 363.4566
Epoch 3/1000
1945/1945 [==============================] - 4s 2ms/step - loss: 311.8925 - val_loss: 243.9465
Epoch 4/1000
1945/1945 [==============================] - 4s 2ms/step - loss: 198.8575 - val_loss: 165.2348
Epoch 5/1000
1945/1945 [==============================] - 4s 2ms/step - loss: 134.1387 - val_loss: 127.1467
Epoch 6/1000
1945/1945 [==============================] - 4s 2ms/step - loss: 105.2458 - val_loss: 87.2645
Epoch 7/1000
1945/1945 [==============================] - 4s 2ms/step - loss: 94.3872 - val_loss: 79.9416
Epoch 8/1000
1945/1945 [==============================] - 4s 2ms/step - loss: 93.3767 - val_loss: 89.1381
Epoch 9/1000
1945/1945 [==============================] - 4s 2ms/step - loss: 84.7895 - val_l

In [81]:
data_df.values

array([[ 1.10700000e+03,  1.12095000e+03,  1.09600000e+03, ...,
         8.21204783e+04,  8.55922243e+01,  4.12845890e+01],
       [ 1.11000000e+03,  1.12460000e+03,  1.08050000e+03, ...,
         6.69971439e+04,  6.89229811e+01,  4.14856898e+01],
       [ 1.10990000e+03,  1.11980000e+03,  1.09505000e+03, ...,
         9.84806940e+04,  7.12061468e+01,  4.02902834e+01],
       ...,
       [ 2.62475000e+03,  2.67600000e+03,  2.60030000e+03, ...,
         5.90741157e+04, -9.64145017e+01,  6.98014234e+01],
       [ 2.62615000e+03,  2.66690000e+03,  2.61205000e+03, ...,
         7.36609720e+04, -9.08103266e+01,  6.87334646e+01],
       [ 2.64500000e+03,  2.65900000e+03,  2.60700000e+03, ...,
         1.96081558e+05, -7.55891915e+01,  6.75382171e+01]])

In [171]:
X_train = X_train.reshape(X_train.shape[0],21,21,1).shape

In [172]:
X_test = X_test.reshape(X_test.shape[0],21,21,1).shape